In [ ]:
import pandas as pd
import os
from tqdm import tqdm

def filter_codes(code):
    return str(code).startswith(('31', '23', '11'))

base_path: str = r"C:\Users\kctco\Documents\Code\seoul-communitydetection\2023\생활이동_자치구_2023"
all_data = pd.DataFrame()

for month in tqdm(range(1, 13), desc="Processing months"):
    month_str = f"{month:02}"
    month_path = base_path + f"{month_str}"

    for file_name in os.listdir(month_path):
        file_path = os.path.join(month_path, file_name)
        if file_name.endswith('.csv'):
            try:
                data = pd.read_csv(file_path, usecols=["출발 시군구 코드", "도착 시군구 코드", "이동인구(합)"], encoding="EUC-KR")
                data['이동인구(합)'] = pd.to_numeric(data['이동인구(합)'].replace("*", pd.NA), errors='coerce').fillna(0)
                
                data = data[data['출발 시군구 코드'].apply(filter_codes) & data['도착 시군구 코드'].apply(filter_codes)]
                
                data = data.groupby(['출발 시군구 코드', '도착 시군구 코드'], as_index=False)['이동인구(합)'].sum()
                
                all_data = pd.concat([all_data, data], ignore_index=True)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

all_data = all_data.groupby(['출발 시군구 코드', '도착 시군구 코드'], as_index=False)['이동인구(합)'].sum()

all_data.to_csv('all_collected_data.csv', index=False)
print("All data saved to 'all_collected_data.csv'.")
